# Discover Sentinel-1 InSAR pairs

- The provider used for this notebook is `cop_dataspace`
- A free account needs to be created at https://dataspace.copernicus.eu/
- The credentials need to be configured as explained here https://eodag.readthedocs.io/en/stable/getting_started_guide/configure.html

In [1]:
%load_ext autoreload
%autoreload 2

from eo_tools.util import explore_products
import geopandas as gpd
import folium
import folium.plugins
import logging

from eodag import EODataAccessGateway

# credentials need to be stored in the following file (see EODAG docs)
confpath = "/data/eodag_config.yml"
dag = EODataAccessGateway(user_conf_file_path=confpath)
# make sure cop_dataspace will be used 
dag.set_preferred_provider("cop_dataspace")
logging.basicConfig(level=logging.INFO)

## Select an area of interest

You may run the cell below and use your own AOI or skip to the next section and use the provided sample AOI.

- Select a drawing tool (polygon, circle or rectangle)
- Draw the AOI on the map
- Click "Export" on the right to save the GeoJSON file

In [ ]:
map_select = folium.Map()
folium.plugins.Draw(
    export=True,
    draw_options={
        "polyline": False,
        "circlemarker": False,
        "marker": False,
    },
).add_to(map_select)
map_select

## Search
Here, you can replace the `file_aoi` path with your own AOI and change the dates to your convenience or use the ones provided by default.

In [ ]:
# load a geometry
file_aoi = "../data/Morocco_AOI.geojson"
shp = gpd.read_file(file_aoi).geometry[0]
search_criteria = {
    "productType": "S1_SAR_SLC",
    "start": "2023-09-01",
    "end": "2023-09-20",
    "geom": shp
}
results, _ = dag.search(**search_criteria)

## Visualize by groups

- For InSAR, products must be overlapping and from the same relative orbit.
- They also have to be of the same ascending / descending configuration.
- The `explore_products` function may be used to visually find such products. 
    - It groups the compatible footprints together.
    - These different groups can be hidden for better visibility by hovering on the top right icon.
    - Hovering on a footprint displays product indices and dates to help selecting relevant data.

In [3]:
explore_products(results, shp)

## Download

In [ ]:
# fill indices of products to download
indices_to_download = [1, 0]
to_download = [results[i] for i in indices_to_download]

# if the data has already been downloaded, just re-run the cell to get the product paths
dl = dag.download_all(to_download, outputs_prefix="/data/S1/", extract=False)

After downloading the products, you can use the `s1-easy-tops-insar` notebook to perform interferometric processing.